In [1]:
cd ../../data/

D:\OneDrive - mail.bnu.edu.cn\work\data


In [2]:
from astropy.io import fits

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table

import numpy as np

1. 读取fits文件信息
2. 制定超新星遗迹范围，并截取
3. 保存超新星遗迹数据

In [3]:
hdulist = fits.open('catwithdis.fit')

In [5]:
hdulist.info()

Filename: catwithdis.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     63   28271752R x 23C   [D, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, 19A, E, E, D, D, D, E]   


In [4]:
cols = hdulist[1].columns
cols

ColDefs(
    name = 'RA'; format = 'D'
    name = 'DEC'; format = 'D'
    name = 'J'; format = 'E'
    name = 'H'; format = 'E'
    name = 'K'; format = 'E'
    name = 'EJ'; format = 'E'
    name = 'EH'; format = 'E'
    name = 'EK'; format = 'E'
    name = 'GP'; format = 'E'
    name = 'BP'; format = 'E'
    name = 'RP'; format = 'E'
    name = 'EGP'; format = 'E'
    name = 'EBP'; format = 'E'
    name = 'ERP'; format = 'E'
    name = 'W1'; format = 'E'
    name = 'EW1'; format = 'E'
    name = 'SSID'; format = '19A'
    name = 'PARA'; format = 'E'
    name = 'EPARA'; format = 'E'
    name = 'REGK'; format = 'D'
    name = 'REBR'; format = 'D'
    name = 'REHK'; format = 'D'
    name = 'DIS'; format = 'E'
)

In [5]:
%%time 

data = hdulist[1].data

Wall time: 0 ns


In [6]:
data

FITS_rec([(74.840139, 26.242048, 12.699, 11.807, 11.565, 0.018, 0.021, 0.018, 15.79435 , 17.155628, 14.638269, 0.00213992, 0.03298068, 0.00830059, 11.359, 0.024, '153696882156882816', 0.4023, 0.0721, 2.05939886, 1.27769975, 0.13008419, 2.2588806 ),
          (74.808284, 26.272211, 11.942, 11.26 , 10.998, 0.019, 0.024, 0.018, 14.982522, 15.842951, 13.622395, 0.0030209 , 0.03750518, 0.01162324, 10.809, 0.023, '153697225754268672', 2.9086, 0.1173, 2.0725545 , 0.82196002, 0.1968211 , 0.34125647),
          (74.837379, 26.293083, 12.994, 12.354, 12.111, 0.018, 0.023, 0.016, 15.747385, 16.90917 , 14.666429, 0.00216768, 0.02942837, 0.01042682, 11.966, 0.023, '153697298769173248', 1.7082, 0.0625, 2.20073703, 1.36746518, 0.16439033, 0.5765563 ),
          ...,
          (90.014229, 43.881161, 13.888, 13.213, 12.954, 0.023, 0.024, 0.024, 16.386814, 17.563307, 15.322656, 0.00529872, 0.05787191, 0.01252781, 12.867, 0.025, '961366719119831424', 2.8186, 0.0728, 0.23149309, 0.14247835, 0.06931999, 0.

In [17]:
header = fits.getheader('catwithdis.fit',1)
header

XTENSION= 'BINTABLE'           /Binary table written by MWRFITS v1.6a           
BITPIX  =                    8 /Required value                                  
NAXIS   =                    2 /Required value                                  
NAXIS1  =                  127 /Number of bytes per row                         
NAXIS2  =             28271752 /Number of rows                                  
PCOUNT  =                    0 /Normally 0 (no varying arrays)                  
GCOUNT  =                    1 /Required value                                  
TFIELDS =                   23 /Number of columns in table                      
COMMENT                                                                         
COMMENT  *** End of mandatory fields ***                                        
COMMENT                                                                         
COMMENT                                                                         
COMMENT  *** Column names **

In [18]:
header = fits.getheader('catwithdis.fit',0)
header

SIMPLE  =                    T /Dummy Created by MWRFITS v1.6a                  
BITPIX  =                    8 /Dummy primary header created by MWRFITS         
NAXIS   =                    0 /No data is associated with this header          
EXTEND  =                    T /Extensions may (will!) be present               

In [ ]:
# hdulist.close()

In [5]:
data=fits.getdata('catwithdis.fit')

In [7]:
%%time

c = SkyCoord(data['ra'] * u.deg, data['dec'] * u.deg, frame='icrs')

Wall time: 19.4 s


In [14]:
%%time

l = c.galactic.l.degree
b = c.galactic.b.degree

Wall time: 23.2 s


In [ ]:
col_l = fits.Column(name='l', format='D', array=l)
col_b = fits.Column(name='b', format='D', array=b)
new_cols = fits.ColDefs([col_l, col_b])
orig_cols = data.columns
hdu = fits.BinTableHDU.from_columns(orig_cols + new_cols)
hdu.writeto('newtable.fits')